# Benchmarking of feature extraction modules

In [2]:
# Some common packages used throughout the notebook
%load_ext autoreload
%autoreload 2

# Import the NumPy module
import numpy as np

# Convient for outputting dictionaries
import json

import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
from satsense.util import load_from_file, normalize_image, get_rgb_bands

# Supported image formats include RGB, Quickbird and Worldview
from satsense.util import RGB, QUICKBIRD, WORLDVIEW2

# URI to the image
imagefile = '/home/bweel/Documents/projects/dynaslum/data/satelite/056239125010_01/056239125010_01_P001_MUL/08NOV02054348-M2AS_R1C1-056239125010_01_P001.TIF'
# Set the correct format here, it is used throughout the notebook
bands = QUICKBIRD

# Loading the file
dataset, image = load_from_file(imagefile)
normalized_image = normalize_image(image, bands)

## Image Utilities

In [4]:
import time
from satsense.util import get_ubyte_image, get_grayscale_image
from satsense.extract import CellGenerator

generator = CellGenerator(normalized_image, 25)

difference = 0
number = 10000
for i in range(number):
    win = generator.next()

    t0 = time.clock()
    get_grayscale_image(win.window, bands=bands)
    t1 = time.clock()

    difference += t1 - t0

difference *= 1000000
print("get_grayscale_image took on average", difference / number, "microseconds")

get_grayscale_image took on average 25.874199999996517 microseconds


In [5]:
import time
from satsense.util import get_ubyte_image, get_grayscale_image
from satsense.extract import CellGenerator

generator = CellGenerator(normalized_image, 25)

difference = 0
number = 10000
for i in range(number):
    win = generator.next()

    t0 = time.clock()
    gray_ubyte = get_ubyte_image(get_grayscale_image(win.window, bands=bands))
    t1 = time.clock()

    difference += t1 - t0

difference *= 1000000
print("get_ubyte_image took on average", difference / number, "microseconds")

get_ubyte_image took on average 83.61029999999924 microseconds


## Cell Generator

In [6]:
import time
from satsense.extract import CellGenerator

generator = CellGenerator(normalized_image, 25)

difference = 0
number = 10000
for i in range(number):
    t0 = time.clock()
    cell = generator.next()
    t1 = time.clock()

    difference += t1 - t0

difference *= 1000000
print("CellGenerator took on average", difference / number, "microseconds")

CellGenerator took on average 5.259300000001499 microseconds


In [7]:
import time
from satsense.extract import CellGenerator, SuperCell

# Setup
generator = CellGenerator(normalized_image, 25)

for i in range(15):
    cell = generator.next()

# Testing
difference = 0
number = 10000
for i in range(number):
    t0 = time.clock()
    SuperCell(normalized_image, cell, 100)
    t1 = time.clock()

    difference += t1 - t0
    
    cell = generator.next()

difference *= 1000000
print("SuperCell took on average", difference / number, " microseconds")

SuperCell took on average 13.711699999997773  microseconds


## Features

### Pantex

In [10]:
from satsense.features import Pantex

#
generator = CellGenerator(normalized_image, 25)

p = Pantex(windows=(100,))

difference = 0
number = 10000
for i in range(number):
    cell = generator.next()
    t0 = time.clock()
    p(normalized_image, cell, bands=bands)
    t1 = time.clock()

    difference += t1 - t0

difference *= 1000000
print("PanTex took on average", difference / number, " microseconds")

PanTex took on average 9836.829400000204  microseconds


In [39]:
from satsense.features import pantex
from satsense.util import get_ubyte_image, get_grayscale_image
from satsense.extract import CellGenerator, SuperCell

#
generator = CellGenerator(normalized_image, 25)

difference = [0, 0]
number = 10000
for i in range(number):
    t0 = time.clock()
    cell = generator.next()
    win = SuperCell(normalized_image, cell, 100)
    gray_ubyte = get_ubyte_image(get_grayscale_image(win.window, bands=bands))
    t1 = time.clock()
    pantex(gray_ubyte)
    t2 = time.clock()

    difference[0] += t1 - t0
    difference[1] += t2 - t1

difference[0] *= 1000000
difference[1] *= 1000000
    
print("Setup took on average", difference[0] / number, "PanTex took on average", difference[1] / number, " microseconds")

Setup took on average 0.000374921499999607 PanTex took on average 0.009714037600000251  microseconds


In [43]:
difference[0] *= 1000000
difference[1] *= 1000000
    
print("Setup took on average", difference[0] / number, "PanTex took on average", difference[1] / number, " microseconds")

Setup took on average 0.000374921499999607 PanTex took on average 0.009714037600000251  microseconds


In [44]:
difference[0] = difference[0]
difference[1] = difference[1]
runtime = (difference[0] + difference[1]) / number
runtime = runtime * 163 * 163 * 3

print("expected runtime is:", runtime, "seconds")

expected runtime is: 804.1606629836887 seconds


In [38]:
from satsense.features.glcm import get_rii_dist_angles

t0 = time.clock()
get_rii_dist_angles()
t1 = time.clock()
difference = t1 - t0
print("Rii dist angles first call took", difference * 1000000 , " microseconds")

difference = 0
number = 1000000
for i in range(number):
    t0 = time.clock()
    get_rii_dist_angles()
    t1 = time.clock()

    difference += t1 - t0

difference *= 1000000
print("Rii dist angles took on average", difference / number, " microseconds")

Rii dist angles first call took 109.00000000901855  microseconds
Rii dist angles took on average 0.7185399999933679  microseconds


### NDVI

In [13]:
from satsense.features import NirNDVI

#
generator = CellGenerator(normalized_image, 25)

ndvi = NirNDVI(windows=(25,))

difference = 0
number = 10000
for i in range(number):
    cell = generator.next()
    t0 = time.clock()
    ndvi(normalized_image, cell, bands=bands)
    t1 = time.clock()

    difference += t1 - t0

difference *= 1000000
print("NirNDVI took on average", difference / number, " microseconds")

NirNDVI took on average 39.501200000145786  microseconds


In [14]:
runtime = difference / number
runtime /= 1000000
runtime = runtime * 163 * 163 * 3

print("expected runtime is:", runtime, "seconds")

expected runtime is: 3.1485221484116197 seconds


## HoG

In [4]:
import time
from satsense.extract import CellGenerator
from satsense.features import HistogramOfGradients

generator = CellGenerator(normalized_image, 25)

hog = HistogramOfGradients(windows=(25,))
difference = 0
number = 1000
for i in range(number):
    win = generator.next()

    t0 = time.clock()
    hog(normalized_image, win, bands=bands)
    t1 = time.clock()

    difference += t1 - t0

difference *= 1000
print("HistogramOfGradients took on average", difference / number, "milliseconds")

HistogramOfGradients took on average 9.055823000000041 milliseconds


In [5]:
runtime = difference / number
runtime /= 1000
runtime = runtime * 163 * 163 * 3

print("expected runtime is:", runtime, "seconds")

expected runtime is: 721.8124838610033 seconds
